In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

In [3]:
prosperLoan = pd.read_csv("prosperLoanData.csv")
FED_data = pd.read_csv("FEDFUNDS.csv")


In [4]:
duplicate_Listing_Key = prosperLoan[
    prosperLoan.duplicated(subset='ListingKey', keep=False)
]
duplicate_Listing_Key = duplicate_Listing_Key.sort_values('ListingKey')
prosperLoan = prosperLoan.drop_duplicates(subset='ListingKey', keep='first')
duplicate_Loan_Key = prosperLoan[
    prosperLoan.duplicated(subset='LoanKey', keep=False)
]

In [5]:
prosperLoan_filtered =  prosperLoan[['LoanOriginationDate',   
    'BorrowerRate' ]]

In [6]:
prosperLoan_filtered['LoanOriginationDate'] = pd.to_datetime(prosperLoan_filtered['LoanOriginationDate'])


C:\Users\sayan\AppData\Local\Temp\ipykernel_4964\3401869797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prosperLoan_filtered['LoanOriginationDate'] = pd.to_datetime(prosperLoan_filtered['LoanOriginationDate'])


In [7]:
prosper_Loan_processed = prosperLoan_filtered.sort_values(by='LoanOriginationDate')

In [8]:
prosper_Loan_processed['Month'] = prosper_Loan_processed['LoanOriginationDate'].dt.to_period('M')
aggregations = {
    'BorrowerRate': ['mean']
}

In [9]:
prosperLoan_monthly = prosper_Loan_processed.groupby('Month', as_index = False).agg(aggregations)


In [10]:
prosperLoan_monthly.columns = ['_'.join(col).strip() for col in prosperLoan_monthly.columns.values]

In [11]:
prosperLoan_monthly.columns


Index(['Month_', 'BorrowerRate_mean'], dtype='object')

In [12]:
prosperLoan_monthly.head()

,Month_,BorrowerRate_mean
0,2005-11,0.109462
1,2005-12,0.070939
2,2006-01,0.084375
3,2006-02,0.128436
4,2006-03,0.171579


In [13]:
prosperLoan_monthly = prosperLoan_monthly.rename(columns={ 'Month_' : 'Month'})

In [14]:
prosperLoan_monthly['Month'].dtype

period[M]

In [15]:
tbill = pd.read_csv("tbill_train.csv")

In [16]:
tbill.rename(columns={'observation_date' : 'Date','TB3MS': 'Treasury_data'}, inplace=True)
tbill.head()

,Date,Treasury_data
0,2001-01-01,5.15
1,2001-02-01,4.88
2,2001-03-01,4.42
3,2001-04-01,3.87
4,2001-05-01,3.62


In [17]:
FED_data.rename(columns={'observation_date' : 'Date','FEDFUNDS': 'fed_rate'}, inplace=True)
FED_data.head()

,Date,fed_rate
0,1954-07-01,0.80
1,1954-08-01,1.22
2,1954-09-01,1.07
3,1954-10-01,0.85
4,1954-11-01,0.83


In [18]:

prosperLoan_monthly = prosperLoan_monthly.sort_values("Month").reset_index(drop=True)

first_loan_month = prosperLoan_monthly["Month"].min()
last_loan_month  = prosperLoan_monthly["Month"].max()

In [19]:
start_month = first_loan_month - 12
end_month   = last_loan_month

In [20]:
tbill["Date"] = pd.to_datetime(tbill["Date"])
tbill["Month"] = tbill["Date"].dt.to_period("M")

In [21]:
tbill = tbill[['Month', 'Treasury_data']]
tbill.columns

Index(['Month', 'Treasury_data'], dtype='object')

In [22]:
FED_data["Date"] = pd.to_datetime(FED_data["Date"])
FED_data["Month"] = FED_data["Date"].dt.to_period("M")



In [23]:
FED_data = FED_data[['Month','fed_rate' ]]
FED_data.columns

Index(['Month', 'fed_rate'], dtype='object')

In [24]:
combined = pd.DataFrame({"Month": pd.period_range(start=start_month, end=end_month, freq="M")})

In [25]:

combined = (combined
         .merge(tbill, on="Month", how="left")
         .merge(FED_data, on="Month", how="left")
         .sort_values("Month")
         .reset_index(drop=True)
        )

In [26]:
print("Missing Treasury months:", combined["Treasury_data"].isna().sum())
print("Missing Fed months:", combined["fed_rate"].isna().sum())

Missing Treasury months: 0
Missing Fed months: 0


In [27]:

lags = list(range(1, 4))  
for k in lags:
    combined[f"Treasury_lag{k}"] = combined["Treasury_data"].shift(k)
    combined[f"Fed_lag{k}"] = combined["fed_rate"].shift(k)

In [28]:
combined.columns

Index(['Month', 'Treasury_data', 'fed_rate', 'Treasury_lag1', 'Fed_lag1',
       'Treasury_lag2', 'Fed_lag2', 'Treasury_lag3', 'Fed_lag3'],
      dtype='object')

In [29]:

lag_cols = [f"Treasury_lag{k}" for k in lags] + [f"Fed_lag{k}" for k in lags]
combined_lagged = combined[["Month"] + lag_cols].copy()

In [30]:
# Keep only Month + lag features 
lag_cols = [f"Treasury_lag{k}" for k in lags] + [f"Fed_lag{k}" for k in lags]
combined_lagged = combined[["Month"] + lag_cols].copy()

In [31]:
# Merge into prosperLoan_monthly and drop rows without all lags

prosperLoanFEDTbill = (prosperLoan_monthly
            .merge(combined_lagged, on="Month", how="left")
            .sort_values("Month")
            .reset_index(drop=True)
           )

In [32]:
prosperLoanFEDTbill.columns

Index(['Month', 'BorrowerRate_mean', 'Treasury_lag1', 'Treasury_lag2',
       'Treasury_lag3', 'Fed_lag1', 'Fed_lag2', 'Fed_lag3'],
      dtype='object')

In [33]:
# Time-based 75/25 train-test split (no leakage)

n = len(prosperLoanFEDTbill)
split_idx = int(n * 0.75)

train = prosperLoanFEDTbill.iloc[:split_idx].copy()
test  = prosperLoanFEDTbill.iloc[split_idx:].copy()

X_train = train[lag_cols]
y_train = train["BorrowerRate_mean"]

X_test  = test[lag_cols]
y_test  = test["BorrowerRate_mean"]

# Quick sanity checks
print("Train months:", train["Month"].min(), "->", train["Month"].max(), " | rows:", len(train))
print("Test months: ", test["Month"].min(),  "->", test["Month"].max(),  " | rows:", len(test))
print("Any overlap?", train["Month"].max() >= test["Month"].min())

Train months: 2005-11 -> 2012-03  | rows: 70
Test months:  2012-04 -> 2014-03  | rows: 24
Any overlap? False


In [34]:
#  Save CSVs
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)